In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import Necessary Libararies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

Read the dataset

In [ ]:
df=pd.read_csv("/kaggle/input/smartphone-sales/smartphones.csv")

e
Dataset First View

In [ ]:
df.head(7)

In [ ]:
df.tail()

Dataset Rows & Columns count

In [ ]:
print('Dataset Rows : {} \n Dataset Columns : {}'.format(df.shape[0],df.shape[1]))

Dataset Information

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

No duplicate values in the dataset

In [ ]:
df.isna().sum()

The dataset has missing values in RAM and Storage

Since RAM and Storage has determined values , we can mode computation

In [ ]:
#Mode Imputation
df['RAM'].fillna(df['RAM'].mode()[0], inplace=True)
df['Storage'].fillna(df['Storage'].mode()[0], inplace=True)

In [ ]:
numerical_features = df.dtypes[df.dtypes != "object"].index
print("Number of Numerical features: ", len(numerical_features))
print(numerical_features)
# Pulling out names of categorical variables by conditioning dtypes equal to object type
categorical_features = df.dtypes[df.dtypes == "object"].index
print("Number of Categorical features: ", len(categorical_features))
print(categorical_features)

In [ ]:

# Dataset Describe
df.describe().T

In [ ]:
df.head()

In [ ]:
df.drop('Smartphone',axis=1,inplace=True)

In [ ]:
# Check Unique Values for each variable.
print(df.apply(lambda col: col.unique())) 

In [ ]:
#visvualization
numerical_cols = df.select_dtypes(include=[float, int])
for col in numerical_cols.columns:
    plt.figure(figsize=(12, 8))
    sns.histplot(data=df, x=col, bins=30, alpha=0.8,kde=True)
    plt.title(f'Histogram of {col}')
    plt.show()

In [ ]:
num_data=df.select_dtypes('number')
corr_matrix=num_data.corr()
corr_matrix

In [ ]:
#check outliers using box plot
for col in numerical_cols:
    plt.figure(figsize=(12, 8))
    plt.boxplot(df[col].dropna())
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)
    plt.show()

Outliers are present in the Price , need to remove them using IQR method

In [ ]:
num_cols=['Storage','Final Price']
for col in num_cols:
    #Check to make sure the column is numeric
    if pd.api.types.is_numeric_dtype(df[col]):
        Q1 = df[col].quantile(.25)
        Q3 = df[col].quantile(.75)
        IQR = Q3 - Q1
        Lower_Fence = Q1 - 1.5 * IQR
        Upper_Fence = Q3 + 1.5 * IQR
        Lower_Outliers = df[df[col] < Lower_Fence][col].values
        Upper_Outliers = df[df[col] > Upper_Fence][col].values
        df[col] = df[col].where(df[col] >= Lower_Fence, Lower_Fence)
        df[col] = df[col].where(df[col] <= Upper_Fence, Upper_Fence)

In [ ]:
num_cols = ['Storage','Final Price']
plt.figure(figsize=(8, 1))
for col in num_cols:
    sns.boxplot(df[col], orient="h")
    plt.title(f"{col} boxplot")
    plt.show()

Outliers has been successfully removed

In [ ]:
df.head()

In [ ]:
df['Brand'].value_counts()

Column Descriptions
1.Smartphone: A detailed description of the smartphone's full name, model, and specifications (such as RAM, storage capacity).

2.Brand: The manufacturer brand of the smartphone (e.g., Realme, Samsung, Motorola).

3.Model: The model name or number of the smartphone (e.g., C55, Galaxy M23).

4.RAM: The amount of RAM (in GB) that the smartphone has, which affects its multitasking performance.

5.Storage: The internal storage capacity of the smartphone (in GB). This represents the space available for storing apps, photos, and other files.

6.Color: The color of the smartphone (e.g., Yellow, Blue, Gray, White).

7.Free: Information indicating whether the smartphone is tied to a carrier. "Yes" means the phone is "SIM-free" or unlocked, allowing it to be used with any carrier.

8.Final Price: The final sale price of the smartphone, representing the ultimate cost after discounts or promotions.

In [ ]:
#distribution of brand
plt.figure(figsize=(30,8))
sns.countplot(data=df,x='Brand')
plt.title("Distribution of Brand")

In [ ]:
#RAM vs final Price
plt.figure(figsize=(12,8))
sns.scatterplot(data=df,x='RAM',y='Final Price')
plt.title("RAM vs Price")

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(data=df,x='RAM',y='Storage')
plt.title("RAM vs Storage")

In [ ]:
number_of_sld_brd = df.groupby("Brand").count().iloc[:,1].reset_index(name = "count").sort_values(by="count",ascending=False)

number_of_sld_brd

In [ ]:
top_5 = number_of_sld_brd[0:6]

top_5

In [ ]:
plt.figure(figsize=(8, 8))

plt.pie(top_5["count"],labels=top_5["Brand"],autopct='%1.1f%%', startangle=90  )               

plt.title("Top 5 Brands by Count")

plt.show()

Top 10 Models

In [ ]:
number_of_sld_model = df.groupby("Model").count().iloc[:,1].reset_index(name = "count").sort_values(by="count",ascending=False)

top_10_model = number_of_sld_model[0:11]

In [ ]:
plt.figure(figsize=(9, 5))

sns.barplot(x="Model", y="count", data=top_10_model,palette="tab10")            

plt.title("Top 10 Model by Count")

plt.xticks(rotation=90)

plt.show()

Brands by AVG Final Price

In [ ]:
grp_price = df.groupby("Brand")["Final Price"].mean().reset_index(name = "mean").sort_values(by="mean",ascending=False)

In [ ]:
plt.figure(figsize=(9,5))

sns.barplot(x="Brand",y="mean",data=grp_price, palette = "Set2")

plt.xticks(rotation=90)

plt.title("Brands by AVG Final Price")

plt.show()

Display Colors of Phones

In [ ]:
colors = df.groupby("Color").count().iloc[:,1].reset_index(name = "count").sort_values(by="count",ascending=False)

In [ ]:
plt.figure(figsize=(9,5))

sns.barplot(x="count",y="Color",data=colors, palette = "Set2")

plt.title("Colors of Phones")

plt.show()

Display the relationship between RAM-final_price and storage-final_price.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

sns.scatterplot(x="RAM",y="Final Price" ,data=df ,ax=axes[0])

sns.scatterplot(x="Storage",y="Final Price" ,data=df,  ax=axes[1])

plt.tight_layout()

plt.show()

In [ ]:
df.to_csv("phone_sales_clean_data.csv")